# Split text from markdown file and store chunks to PostgreSQL DB

In [ ]:
import pandas as pd

file_path = "path_to_the_file"
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
import re
from typing import List, Dict

def split_text_into_chunks(text: str, max_tokens: int = 250) -> List[str]:
    paragraphs = re.split(r'(?:\n\s*\n|</p>\s*<p>)', text.strip())
    chunks, current = [], []

    def token_len(s):
        return len(s.split())

    for para in paragraphs:
        if not para.strip():
            continue
        if token_len(" ".join(current) + " " + para) > max_tokens:
            if current:
                chunks.append(" ".join(current).strip())
                current = []
        current.append(para)
    if current:
        chunks.append(" ".join(current).strip())
    return chunks


def preprocess_pdf_text(document_text: str, max_tokens: int = 250) -> List[Dict]:
    pages = re.split(r'\n\s*---\s*\n', document_text.strip())
    all_chunks = []

    for page_number, page_text in enumerate(pages, start=1):
        pos = 0
        for match in re.finditer(r'<table.*?>.*?</table>', page_text, re.DOTALL | re.IGNORECASE):
            start, end = match.span()

            # text before the table -> normal chunks
            before = page_text[pos:start].strip()
            if before:
                for chunk in split_text_into_chunks(before, max_tokens=max_tokens):
                    all_chunks.append({
                        "page_number": page_number,
                        "content": chunk
                    })

            # table block -> keep as is
            table_block = match.group(0).strip()
            all_chunks.append({
                "page_number": page_number,
                "content": table_block
            })

            pos = end

        # text after the last table
        after = page_text[pos:].strip()
        if after:
            for chunk in split_text_into_chunks(after, max_tokens=max_tokens):
                all_chunks.append({
                    "page_number": page_number,
                    "content": chunk
                })

    return all_chunks

In [ ]:
chunks = preprocess_pdf_text(text, max_tokens=1000)

for c in chunks:
    print(f"Page {c['page_number']}\n{c['content']}\n")

In [ ]:
df = pd.DataFrame(chunks)

df["filename"] = "filename"
df = df[["filename", "page_number", "content"]]

In [ ]:
from FlagEmbedding import BGEM3FlagModel
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True, device=device)

embeddings = []
batch_size = 12

texts = df["content"].tolist()

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
    batch = texts[i:i+batch_size]
    batch_embeddings = model.encode(batch, batch_size=len(batch), max_length=8192)["dense_vecs"]
    embeddings.extend(batch_embeddings.tolist())

df["embedding"] = embeddings

In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 5432))
}

try:
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()
    print("Connected successfully!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS chunks_table (
        id SERIAL PRIMARY KEY,
        "filename" TEXT,
        "page_number" SMALLINT,
        "content" TEXT,
        "embedding" VECTOR(1024)
    )
""")
print("Table is created or already exists")
conn.commit()

In [ ]:
from psycopg2.extras import execute_batch
import ast

df["embedding"] = df["embedding"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

insert_query = """
INSERT INTO chunks_table (
    "filename", "page_number", "content",
    "embedding"
) VALUES (%s, %s, %s, %s);
"""

records = [
    (
        row["filename"],
        row["page_number"],
        row["content"],
        row["embedding"]
    )
    for _, row in df.iterrows()
]

execute_batch(cursor, insert_query, records)
conn.commit()
print("Done")

# Store structured files to DB

In [ ]:
import os
import re
import pandas as pd

input_path = "path_to_input_folder"

rows = []

for fname in os.listdir(input_path):
    if not fname.endswith(".txt"):
        continue

    base_match = re.match(r"(.+)\.pdf_page(\d+)\.txt", fname)
    if not base_match:
        continue

    pdf_name = base_match.group(1) + ".pdf"
    page_number = int(base_match.group(2))

    with open(os.path.join(input_path, fname), encoding="utf-8") as f:
        text = f.read()

    parts = re.split(r'(?=id:\s*\d+)', text.strip())

    if parts and not parts[0].strip().startswith("id:"):
        header = parts[0].strip()
        if header:
            rows.append({
                "filename": pdf_name,
                "page_number": page_number,
                "content": header
            })
        parts = parts[1:]

    for part in parts:
        if part.strip():
            rows.append({
                "filename": pdf_name,
                "page_number": page_number,
                "content": part.strip()
            })

In [ ]:
from FlagEmbedding import BGEM3FlagModel
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True, device=device)

embeddings = []
batch_size = 12

texts = df["content"].tolist()

for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
    batch = texts[i:i+batch_size]
    batch_embeddings = model.encode(batch, batch_size=len(batch), max_length=8192)["dense_vecs"]
    embeddings.extend(batch_embeddings.tolist())

df["embedding"] = embeddings

In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

DB_PARAMS = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 5432))
}

try:
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()
    print("Connected successfully!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS chunks_table (
        id SERIAL PRIMARY KEY,
        "filename" TEXT,
        "page_number" SMALLINT,
        "content" TEXT,
        "embedding" VECTOR(1024)
    )
""")
print("Table created or already exists")
conn.commit()

In [ ]:
from psycopg2.extras import execute_batch
import ast

df["embedding"] = df["embedding"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

insert_query = """
INSERT INTO chunks_table (
    "filename", "page_number", "content",
    "embedding"
) VALUES (%s, %s, %s, %s);
"""

records = [
    (
        row["filename"],
        row["page_number"],
        row["content"],
        row["embedding"]
    )
    for _, row in df.iterrows()
]

execute_batch(cursor, insert_query, records)
conn.commit()
print("Done")